# The objective is to weave the tweets into conversation threads. File twcs is one tweet per row with link information to next and previous tweets: therefore organized as an Adjacency Matrix.

Import relevant libraries
Pandas for data wrangling and datetime for performance checking

In [8]:
import pandas as pd
import datetime as dt

Import CSV file with one tweet per record
parse the date fields as such

In [9]:
# .py file was run with nrows set to 1 million. Here we will use 10001 for the demonstration

full_df = pd.read_csv("twcs.csv", na_filter= False, nrows=10001,
                      parse_dates = ['created_at'], dtype = {'tweet_id': str,'in_response_to_tweet_id': str, 'inbound':bool, 'response_tweet_id':str })
full_df.head(300)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,2017-10-31 22:10:47+00:00,@115712 I understand. I would like to assist y...,2,3
1,2,115712,True,2017-10-31 22:11:45+00:00,@sprintcare and how do you propose we do that,,1
2,3,115712,True,2017-10-31 22:08:27+00:00,@sprintcare I have sent several private messag...,1,4
3,4,sprintcare,False,2017-10-31 21:54:49+00:00,@115712 Please send us a Private Message so th...,3,5
4,5,115712,True,2017-10-31 21:49:35+00:00,@sprintcare I did.,4,6
...,...,...,...,...,...,...,...
295,403,AskAmex,False,2017-10-30 13:34:01+00:00,@115807 I am so sorry to hear that. Which U.S....,399,404
296,404,115807,True,2017-10-29 23:07:07+00:00,@AskAmex Nope. They weren’t ready to listen wh...,403,405
297,405,AskAmex,False,2017-10-29 21:57:36+00:00,"@115807 Hi there, I am so sorry to her that. w...",404,406
298,406,115807,True,2017-10-29 21:48:00+00:00,@AskAmex Thought Amex values their customers. ...,405,


Create a thread that encompasses all related tweets
The final tweet of each thread is the one with no next-tweet in the response_tweet_id

In [10]:
mask = full_df["response_tweet_id"] == ""
thread = full_df[mask].copy()
thread.drop(["text","inbound","created_at","response_tweet_id","in_response_to_tweet_id"], axis = 1, inplace = True)
thread.head(3)

,tweet_id,author_id
1,2,115712
7,11,sprintcare
13,19,sprintcare


Create the summary fields that will gather the data, for instance tweet_l will be a list of tweet_ids
separated by commas, and verify_time will have a True if the tweets in tweet_l have ascending timestamps

In [11]:
thread["company_name"]=""
thread["tweet_l"] = ""
thread["author_l"] =""
thread["inbound_l"] = ""
thread["time_l"] = ""
thread["length"] = 0
thread["verify_thread"] = False
thread["verify_time"] = False
thread["verify_alternance"] = False
thread["inbound_first"] = False
thread.head(3)

,tweet_id,author_id,company_name,tweet_l,author_l,inbound_l,time_l,length,verify_thread,verify_time,verify_alternance,inbound_first
1,2,115712,,,,,,0,False,False,False,False
7,11,sprintcare,,,,,,0,False,False,False,False
13,19,sprintcare,,,,,,0,False,False,False,False


Indexing the dataframes. I ran into problems when indexing first from the pd.read_csv function.
It automatically used integers despite the field being imported as dtype string

In [12]:
full_df.set_index("tweet_id", inplace = True)
thread.set_index("tweet_id", inplace = True)
print("Length of thread dataframe = ", len(thread))

Length of thread dataframe =  3281


Sequentially process each thread in the thread dataframe
The id of the thread is the last tweet in the thread so it works backwards collecting the related tweets

In [13]:
for row in range(len(thread)):
    thread_end = thread.index[row]
    prev_tweet = thread_end
    
    if row % 10000 == 0:
        print("Processing thread number: ", row , "time: ", dt.datetime.now().time())
    
    tweet_list = []
    author_list = []
    inbound_list = []
    time_list = []

    """
    Follow the tweets that correspond to a thread in the tweets dataframe
    using the previous tweet value, as we are going backwards from the end to the beginning
    """

    consistent_thread = True
    while (prev_tweet != "")  & consistent_thread:
        if (prev_tweet in full_df.index): 
            tweet_list.append(prev_tweet)
            my_list = full_df.loc[prev_tweet,["author_id","created_at","inbound","in_response_to_tweet_id"]]
            author_list.append(my_list[0])
            time_list.append(str(my_list[1]))
            inbound_list.append(str(my_list[2]))
            prev_tweet = my_list[3]
        else:
            consistent_thread = False
            
    """
    If the thread is consistent, meaning that all the tweets in the chain do exist,
    it starts building the lists of tweets, authors, etc. to write them in the appropriate fields of the thread dataframe
    """

    if consistent_thread:
        tweet_l       = '|'.join(tweet_list[::-1])
        author_l      = '|'.join(author_list[::-1])
        inbound_l     = '|'.join(inbound_list[::-1])
        time_l        = '|'.join(time_list[::-1])
        length        = len(tweet_list) 
        verify_thread = True
        inbound_first = inbound_list[-1]  
        verify_time   = True
        verify_alternance = True
        
        if "False" in inbound_list:
            company_name = author_list[inbound_list.index("False")]
        else:
            company_name = ""        
        if len(tweet_list) > 1:
            for i in range(len(tweet_list)-1):
                verify_alternance = verify_alternance & (inbound_list[i] != inbound_list[i+1])
                verify_time = verify_time & (time_list[i]>= time_list[i+1])
                
        thread.loc[thread_end, ["company_name","tweet_l","author_l","inbound_l","time_l","length","verify_thread","inbound_first","verify_time",
                                "verify_alternance"]] = [company_name,tweet_l,author_l,inbound_l,time_l,length,verify_thread,inbound_first,
                                                        verify_time, verify_alternance]  

Processing thread number:  0 time:  23:41:28.519358


In [14]:
thread.head(4)

,author_id,company_name,tweet_l,author_l,inbound_l,time_l,length,verify_thread,verify_time,verify_alternance,inbound_first
tweet_id,,,,,,,,,,,
2,115712,sprintcare,8|6|5|4|3|1|2,115712|sprintcare|115712|sprintcare|115712|spr...,True|False|True|False|True|False|True,2017-10-31 21:45:10+00:00|2017-10-31 21:46:24+...,7,True,True,True,True
11,sprintcare,sprintcare,18|17|16|15|12|11,115713|sprintcare|115713|sprintcare|115713|spr...,True|False|True|False|True|False,2017-10-31 19:56:01+00:00|2017-10-31 19:59:13+...,6,True,True,True,True
19,sprintcare,sprintcare,20|19,115715|sprintcare,True|False,2017-10-31 22:03:34+00:00|2017-10-31 22:10:10+...,2,True,True,True,True
27,Ask_Spectrum,Ask_Spectrum,29|28|24|21|22|25|26|27,115716|Ask_Spectrum|115716|Ask_Spectrum|115716...,True|False|True|False|True|False|True|False,2017-10-31 22:01:35+00:00|2017-10-31 22:05:37+...,8,True,True,True,True


In [ ]:
thread.to_csv(r'threads.csv')  